In [1]:
from scipy.spatial import distance as dist
import numpy as np
import cv2
from imutils import face_utils
from imutils.video import VideoStream
import imutils
from fastai.vision import *
import argparse
import time
import dlib
from playsound import playsound
from torch.serialization import SourceChangeWarning
warnings.filterwarnings("ignore", category=SourceChangeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
path = './'
print(path + 'export.pkl')
learn = load_learner(path, 'export.pkl')

./export.pkl


In [3]:
face_cascade = cv2.CascadeClassifier("../haarcascade_frontalface_alt2.xml")
vs = VideoStream(src=0).start()
start = time.perf_counter()
data = []
time_value = 0
EYE_AR_THRESH = 0.20
EYE_AR_CONSEC_FRAMES = 10
COUNTER = 0
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear
def data_time(time_value, prediction, probability, ear):
    current_time = int(time.perf_counter()-start)
    if current_time != time_value:
        data.append([current_time, prediction, probability, ear])
        time_value = current_time
    return time_value
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [4]:
# Function to test the emotion
def test_emotion(emotion, vs):
    frame = vs.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_coord = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(30, 30))
    return 

# To start the game you will press q. Once pressed the first emotion you will
# make is neutral. If you do make the emotion for 3 seconds you get a correct
# noise and if not it will give an incorrect noise. There will be a noises
# introducing all the emotions and noises to say you passed or you did not pass.
# This will go in order after neutral to happy, sad, surprised, then angry. Once
# you go through all emotions correctly you win.

In [6]:
emotions = ['neutral','happy', 'sad', 'surprise', 'angry'] # put the emotions
FRAMES_TO_PASS = 6
for emotion in emotions:
    emotion_counter = 0
    first_pass = True
    while True:
        frame = vs.read()
        frame = imutils.resize(frame, width=450)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_coord = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(30, 30))
        cv2.putText(frame, f"Make a {emotion} face!", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        for coords in face_coord:
            X, Y, w, h = coords
            H, W, _ = frame.shape
            X_1, X_2 = (max(0, X - int(w * 0.3)), min(X + int(1.3 * w), W))
            Y_1, Y_2 = (max(0, Y - int(0.3 * h)), min(Y + int(1.3 * h), H))
            img_cp = gray[Y_1:Y_2, X_1:X_2].copy()
            prediction, idx, probability = learn.predict(Image(pil2tensor(img_cp, np.float32).div_(225)))
            cv2.rectangle(
                img=frame,
                pt1=(X_1, Y_1),
                pt2=(X_2, Y_2),
                color=(128, 128, 0),
                thickness=2,
            )
            rect = dlib.rectangle(X, Y, X+w, Y+h)
            cv2.putText(frame, str(prediction), (10, frame.shape[0] - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (225, 255, 255), 2)
            cv2.putText(frame, "Press q to quit", (250, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
        if str(prediction) == emotion:
            emotion_counter += 1
        if emotion_counter > FRAMES_TO_PASS:
            playsound('../sounds/correct.mp3')
            cv2.imshow("frame", frame)
            cv2.putText(frame, f"You passed the {emotion} round!", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            print(f'You passed the {emotion} round')
            break
        cv2.imshow("frame", frame)
        if first_pass:
            playsound('../sounds/' + emotion + '.mp3')
            first_pass = False
        if cv2.waitKey(1) & 0xFF == ord("q") :
            vs.stop()
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            break
# End the game
playsound('../sounds/win.mp3')
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=450)
    cv2.putText(frame, "You win!!!", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q") :
        vs.stop()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        break

You passed the neutral round
You passed the happy round
You passed the sad round
You passed the surprise round
You passed the angry round
